#Optimizing Model Parameters

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



##Hyperparameters
- Hyperparameters are adjustable parameters that let you control the model optimization process.
    - Number of Epochs
    - Batch Size
    - Learning Rate

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 30

##Optimization Loop
- The Train Loop: iterate over the training dataset and try to converge to optimal parameters.
- The Validation/Test Loop: iterate over the test dataset to check if mofel performance is improving.

###Loss Function
- `nn.MSELoss`: Mean Square Error
- `nn.NLLLoss`: Negative Log Likelihood
- `nn.CrossEntropyLoss`: `nn.LogSoftmax` + `nn.NLLLoss`

In [4]:
loss_fn = nn.CrossEntropyLoss()

###Optimizer
- All optimization logic is encapsulated in the `torch.optim`.
- Inside the training loop, optimization happens in three steps.
    1. Call `optimizer.zero_grad()` to reset the gradients of model parameters.
    2. Backpropagate the prediction loss with a call to `loss.backward()`.
    3. Once we have our gradients, we call `optimizer.step()` to adjust the parameters.

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

##Full Implementation

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303867  [    0/60000]
loss: 2.295891  [ 6400/60000]
loss: 2.274737  [12800/60000]
loss: 2.265535  [19200/60000]
loss: 2.248948  [25600/60000]
loss: 2.229021  [32000/60000]
loss: 2.225548  [38400/60000]
loss: 2.199094  [44800/60000]
loss: 2.192761  [51200/60000]
loss: 2.158662  [57600/60000]
Test Error: 
 Accuracy: 47.1%, Avg loss: 2.152020 

Epoch 2
-------------------------------
loss: 2.163567  [    0/60000]
loss: 2.158027  [ 6400/60000]
loss: 2.095992  [12800/60000]
loss: 2.110598  [19200/60000]
loss: 2.068403  [25600/60000]
loss: 2.009265  [32000/60000]
loss: 2.036062  [38400/60000]
loss: 1.964109  [44800/60000]
loss: 1.966967  [51200/60000]
loss: 1.890824  [57600/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 1.888535 

Epoch 3
-------------------------------
loss: 1.926910  [    0/60000]
loss: 1.898922  [ 6400/60000]
loss: 1.773664  [12800/60000]
loss: 1.812648  [19200/60000]
loss: 1.706085  [25600/60000]
loss: 1.661096  [32000/600